In [1]:
import duckdb
print(duckdb.__version__)
import os

1.2.2


In [16]:
# Set DuckDB memory spill directory and max size

#duckdb.sql("SET temp_directory = '/mnt/ssd/tmp/duckdb_swap';")
duckdb.sql("SET max_temp_directory_size = '200GB';")

## Ingest data

In [2]:
y  = "/mnt/ugreen/HuggingFace/Ethereum_blockchain_parquet"
y2 = "/mnt/ugreen/Eth_pq_sample/Ethereum_blockchain_parquet_full_zstd_10"
y3 = "/mnt/ugreen/Ethereum_blockchain_parquet_full_zstd_10"

def get_blocks_folder(folder):
    return os.path.join(folder, "blocks", "*.parquet")

def get_tx_folder(folder):
    return os.path.join(folder, "transactions", "*.parquet")

In [3]:
def ingest_pq(*cols, folder):

    # Convert column names to a comma-separated string
    col_str = ", ".join(cols) if cols else "*"

    pq_input = duckdb.sql(f"""
                           SELECT {col_str} 
                           FROM read_parquet('{folder}')
                           """)

    return pq_input

## Run query

### Count

In [52]:
def get_num_rows(folder):

    one_col = ingest_pq("block_number", folder = folder)

    num_rows = duckdb.sql("""
                          SELECT COUNT(*) FROM one_col
                          """)

    print(num_rows)

    return None    

In [53]:
%%time

get_num_rows(get_blocks_folder(y))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       634440 │
└──────────────┘

CPU times: user 32.5 ms, sys: 27.9 ms, total: 60.3 ms
Wall time: 74.7 ms


In [54]:
%%time

get_num_rows(get_tx_folder(y))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     93492934 │
└──────────────┘

CPU times: user 115 ms, sys: 26 ms, total: 141 ms
Wall time: 64.2 ms


### Get time range

In [10]:
def get_time_range(folder):

    one_col = ingest_pq("timestamp", folder = folder)

    min_max_time = duckdb.sql("""
                    SELECT 
                    MIN(TO_TIMESTAMP(timestamp)) FILTER (WHERE timestamp > 0) AS min_time, 
                    MAX(TO_TIMESTAMP(timestamp)) AS max_time
                    FROM one_col   
                    """)

    return min_max_time   

In [11]:
%%time

get_time_range(get_blocks_folder(y))

CPU times: user 1.55 ms, sys: 3 ms, total: 4.55 ms
Wall time: 3.66 ms


┌──────────────────────────┬──────────────────────────┐
│         min_time         │         max_time         │
│ timestamp with time zone │ timestamp with time zone │
├──────────────────────────┼──────────────────────────┤
│ 2016-11-17 01:40:08+01   │ 2025-03-25 22:17:35+01   │
└──────────────────────────┴──────────────────────────┘

### Filter blocks

In [4]:
def get_blocks_filtered(t_start, t_end, folder):

    dim_cols = ingest_pq("block_number", "timestamp",
                          folder = folder)

    filter_blocks = duckdb.sql(f"""                                                              
                                SELECT * FROM (
                                  SELECT block_number, TO_TIMESTAMP(timestamp) AS timestamp
                                  FROM dim_cols
                                )
                                WHERE timestamp BETWEEN '{t_start}' AND '{t_end}'
                               """)

    return filter_blocks    

In [5]:
def get_blocks_min_max(filter_blocks):

    block_range = duckdb.sql(f"""
                             SELECT 
                               MIN(block_number) AS min_block,
                               MAX(block_number) AS max_block
                             FROM filter_blocks
                             """).fetchone()

    return block_range    

### Block level analytics

#### Tx count per block

In [30]:
def get_tx_count_per_block(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        folder = get_tx_folder(folder)
                       )
    
    count_per_block = duckdb.sql(f"""
                                    SELECT 
                                        tx.block_number, 
                                        b.timestamp, 
                                        COUNT(*) AS count_tx
                                    FROM (
                                      SELECT block_number, success 
                                      FROM tx_cols
                                      WHERE success IS TRUE
                                      AND block_number >= {min_block}
                                      AND block_number <= {max_block}
                                    ) tx 
                                    INNER JOIN blocks b
                                        ON tx.block_number = b.block_number
                                    GROUP BY 
                                        tx.block_number, 
                                        b.timestamp
                                    ORDER BY count_tx DESC
                                    LIMIT 100
                                 """)       
       
    return count_per_block

In [32]:
%%time

get_tx_count_per_block("2023-01-01", "2025-01-01", y3).df()

CPU times: user 1min 38s, sys: 18.9 s, total: 1min 57s
Wall time: 14.3 s


,block_number,timestamp,count_tx
0,19879530,2024-05-16 04:22:47+02:00,1409
1,19865108,2024-05-14 03:55:35+02:00,1399
2,20566170,2024-08-20 02:11:11+02:00,1396
3,20566207,2024-08-20 02:18:35+02:00,1396
4,19864570,2024-05-14 02:07:23+02:00,1389
...,...,...,...
95,19885302,2024-05-16 23:44:47+02:00,1274
96,19537267,2024-03-29 05:25:23+01:00,1274
97,18330408,2023-10-12 01:16:35+02:00,1273
98,17941432,2023-08-18 13:39:59+02:00,1273


#### Get total ETH per block

In [38]:
def get_eth_per_block(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        "value_f64",
                        folder = get_tx_folder(folder)
                       )
    
    eth_per_block = duckdb.sql(f"""
                                 SELECT 
                                      tx.block_number, 
                                      b.timestamp, 
                                      COUNT(*) AS count_tx,
                                      SUM(tx.value_f64 / 1e18) AS total_tx_eth
                                 FROM (
                                   SELECT block_number, success, value_f64 
                                   FROM tx_cols
                                   WHERE success IS TRUE
                                   AND block_number >= {min_block}
                                   AND block_number <= {max_block}
                                 ) tx 
                                 INNER JOIN blocks b
                                     ON tx.block_number = b.block_number
                                 GROUP BY 
                                     tx.block_number, 
                                     b.timestamp
                                 ORDER BY total_tx_eth DESC
                                 LIMIT 100
                               """)   
       
    return eth_per_block

In [40]:
%%time

get_eth_per_block("2023-01-01", "2025-01-01", y3).df()

CPU times: user 1min 56s, sys: 38 s, total: 2min 34s
Wall time: 39.5 s


,block_number,timestamp,count_tx,total_tx_eth
0,19979969,2024-05-30 05:26:11+02:00,121,550014.467481
1,16601138,2023-02-10 22:47:47+01:00,135,460681.672990
2,17166640,2023-05-01 16:42:23+02:00,130,459342.216039
3,16983245,2023-04-05 16:43:23+02:00,168,454473.409200
4,17475090,2023-06-14 04:00:47+02:00,268,450030.508428
...,...,...,...,...
95,17590152,2023-06-30 07:54:59+02:00,110,121498.033218
96,16808455,2023-03-12 01:40:47+01:00,102,120885.586641
97,19229078,2024-02-14 23:09:23+01:00,152,120342.489459
98,18116584,2023-09-12 02:08:11+02:00,171,120338.212185


#### Get total gas used % per block

In [41]:
def get_gas_per_block(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        "gas_limit",
                        "gas_used",
                        folder = get_tx_folder(folder)
                       )
    
    gas_per_block = duckdb.sql(f"""    
                               SELECT 
                                 tx.block_number, 
                                 b.timestamp, 
                                 SUM(tx.gas_limit) AS sum_gas_limit,
                                 SUM(tx.gas_used) AS sum_gas_used,
                                 (SUM(tx.gas_used) * 100.0 / NULLIF(SUM(tx.gas_limit), 0)) AS gas_pert
                                    
                                FROM (
                                  SELECT *
                                  FROM tx_cols
                                  WHERE success IS TRUE
                                  AND block_number >= {min_block}
                                  AND block_number <= {max_block}
                                ) tx

                                INNER JOIN blocks b
                                    ON tx.block_number = b.block_number
                                    
                                GROUP BY 
                                    tx.block_number, 
                                    b.timestamp
                                -- ORDER BY sum_gas_used DESC
                                -- LIMIT 100;
                                """)    
       
    return gas_per_block

In [42]:
# %%time

# get_gas_per_block("2020-01-01", "2025-01-01", y).df()

#### Get total gas % distribution per block

In [125]:
def calculate_gas_dist(gas_data, bin_size):
    
    gas_dist = duckdb.sql(f"""
                          WITH non_null_gas_pert AS (
                            SELECT gas_pert
                            FROM gas_data
                            WHERE gas_pert IS NOT NULL
                          ),

                          binned AS (
                              SELECT 
                                  FLOOR(gas_pert / {bin_size}) * {bin_size} AS lower_gas_pert
                              FROM non_null_gas_pert
                          ),

                          counted AS (
                              SELECT 
                                  lower_gas_pert,
                                  COUNT(*) AS count_blocks
                              FROM binned
                              GROUP BY lower_gas_pert
                          ),

                          finalized AS (
                              SELECT 
                                  lower_gas_pert,
                                  CASE 
                                      WHEN lower_gas_pert < 100 
                                          THEN lower_gas_pert + {bin_size}
                                      ELSE lower_gas_pert
                                  END AS upper_gas_pert,
                                  count_blocks
                              FROM counted
                          )

                          SELECT *
                          FROM finalized
                          ORDER BY lower_gas_pert DESC;
                          """)
    
    return gas_dist 

In [119]:
def get_gas_dist_per_block(t_start, t_end, bin_size, folder):
    
    gas_per_block = get_gas_per_block(t_start, t_end, folder)
    
    gas_dist = calculate_gas_dist(gas_per_block, bin_size)   
       
    return gas_dist

In [113]:
%%time

get_gas_dist_per_block("2020-01-01", "2025-01-01", 10, y).df()

CPU times: user 7.53 s, sys: 375 ms, total: 7.91 s
Wall time: 411 ms


,lower_gas_pert,upper_gas_pert,count_blocks
0,100.0,100.0,44
1,90.0,100.0,632
2,80.0,90.0,2540
3,70.0,80.0,13101
4,60.0,70.0,46916
5,50.0,60.0,94598
6,40.0,50.0,112950
7,30.0,40.0,79230
8,20.0,30.0,42034
9,10.0,20.0,10183


In [47]:
%%time

get_gas_dist_per_block("2023-01-01", "2025-01-01", 10, y3).df()

CPU times: user 2min 45s, sys: 42.8 s, total: 3min 28s
Wall time: 37.4 s


,lower_gas_pert,upper_gas_pert,count_blocks
0,100.0,100.0,97
1,90.0,100.0,2223
2,80.0,90.0,12093
3,70.0,80.0,72080
4,60.0,70.0,412805
5,50.0,60.0,1188648
6,40.0,50.0,1483939
7,30.0,40.0,1046603
8,20.0,30.0,804269
9,10.0,20.0,183778


### Tx level analytics

#### Get ETH per tx

In [97]:
def get_eth_per_tx(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        "value_f64",
                        "transaction_hash",
                        folder = get_tx_folder(folder)
                       )
    
    eth_per_tx = duckdb.sql(f"""    
                             SELECT 
                               tx.block_number, 
                               b.timestamp, 
                               tx.value_f64 / 1e18 AS tx_eth,
                               '0x' || TO_HEX(tx.transaction_hash) AS transaction_hash_encode                                 
                                 
                             FROM (
                               SELECT *
                               FROM tx_cols
                               WHERE success IS TRUE
                               AND block_number >= {min_block}
                               AND block_number <= {max_block}
                             ) tx
                             
                             INNER JOIN blocks b
                                 ON tx.block_number = b.block_number                                 
                                                       
                             -- ORDER BY tx_eth DESC
                             -- LIMIT 10;
                             """)    
       
    return eth_per_tx

In [87]:
# %%time

# get_eth_per_tx("2023-01-01", "2025-01-01", y3).df()

#### Get gas price per tx

In [94]:
def get_gas_price_per_tx(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        "gas_price",
                        "from_address",
                        "to_address",
                        "transaction_hash",
                        folder = get_tx_folder(folder)
                       )
    
    gas_per_tx = duckdb.sql(f"""    
                             SELECT 
                               tx.block_number, 
                               b.timestamp,
                               -- Wei to Gwei
                               tx.gas_price / 1e9 AS tx_gas_price,
                               '0x' || TO_HEX(tx.transaction_hash) AS transaction_hash_encode,
                               '0x' || TO_HEX(tx.from_address) AS from_address,
                               '0x' || TO_HEX(tx.to_address) AS to_address
                                 
                             FROM (
                               SELECT *
                               FROM tx_cols
                               WHERE success IS TRUE
                               AND block_number >= {min_block}
                               AND block_number <= {max_block}
                             ) tx
                             
                             INNER JOIN blocks b
                                 ON tx.block_number = b.block_number                                 
                                                       
                             ORDER BY tx_gas_price DESC
                             LIMIT 10;
                             """)    
       
    return gas_per_tx

In [96]:
# %%time

# get_gas_price_per_tx("2024-11-01", "2025-01-01", y3).df()

#### Get ETH distribution per tx

In [104]:
def get_eth_dist_per_tx(t_start, t_end, bin_size, folder):
    
    eth_per_tx = get_eth_per_tx(t_start, t_end, folder)
    
    eth_dist = duckdb.sql(f"""
                          WITH non_null_tx_eth AS (
                            SELECT tx_eth
                            FROM eth_per_tx
                            WHERE tx_eth IS NOT NULL
                          ),

                          binned AS (
                              SELECT 
                                  FLOOR(tx_eth / {bin_size}) * {bin_size} AS lower_tx_eth
                              FROM non_null_tx_eth
                          ),

                          counted AS (
                              SELECT 
                                  lower_tx_eth,
                                  COUNT(*) AS count_blocks
                              FROM binned
                              GROUP BY lower_tx_eth
                          ),

                          finalized AS (
                              SELECT 
                                  lower_tx_eth,
                                  CASE 
                                      WHEN lower_tx_eth < 550000 
                                          THEN lower_tx_eth + {bin_size}
                                      ELSE lower_tx_eth
                                  END AS upper_tx_eth,
                                  count_blocks
                              FROM counted
                          )

                          SELECT *
                          FROM finalized
                          ORDER BY lower_tx_eth DESC;
                          """)   
       
    return eth_dist

In [107]:
%%time

get_eth_dist_per_tx("2022-01-01", "2025-01-01", 1000, y3).df()

CPU times: user 2min 45s, sys: 59.9 s, total: 3min 44s
Wall time: 56.8 s


,lower_tx_eth,upper_tx_eth,count_blocks
0,1490000.0,1490000.0,1
1,799000.0,799000.0,1
2,716000.0,716000.0,1
3,584000.0,584000.0,1
4,550000.0,550000.0,1
...,...,...,...
199,4000.0,5000.0,8804
200,3000.0,4000.0,17352
201,2000.0,3000.0,33687
202,1000.0,2000.0,110954


#### Get gas consumed % distribution per tx

In [122]:
def get_gas_dist_per_tx(t_start, t_end, bin_size, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        "gas_limit",
                        "gas_used",
                        folder = get_tx_folder(folder)
                       )
    
    gas_per_tx = duckdb.sql(f"""    
                               SELECT 
                                 tx.block_number,                                
                                 ((tx.gas_used) * 100.0) / NULLIF(tx.gas_limit, 0) AS gas_pert
                                    
                                FROM (
                                  SELECT *
                                  FROM tx_cols
                                  WHERE success IS TRUE
                                  AND block_number >= {min_block}
                                  AND block_number <= {max_block}
                                ) tx
                             """)
    
    gas_dist = calculate_gas_dist(gas_per_tx, bin_size)
       
    return gas_dist

In [124]:
%%time

get_gas_dist_per_tx("2022-01-01", "2025-01-01", 10, y3).df()

CPU times: user 3min 50s, sys: 16.1 s, total: 4min 6s
Wall time: 44.8 s


,lower_gas_pert,upper_gas_pert,count_blocks
0,100.0,100.0,352007404
1,90.0,100.0,105039099
2,80.0,90.0,95240751
3,70.0,80.0,134612803
4,60.0,70.0,160980540
5,50.0,60.0,80346600
6,40.0,50.0,52728038
7,30.0,40.0,41644642
8,20.0,30.0,61694193
9,10.0,20.0,70880600


#### Get gas price distribution per tx

In [129]:
def get_gas_price_dist_per_tx(t_start, t_end, bin_size, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        "gas_price",                        
                        folder = get_tx_folder(folder)
                       )
    
    gas_price_per_tx = duckdb.sql(f"""    
                             SELECT 
                               tx.block_number, 
                               -- Wei to Gwei
                               tx.gas_price / 1e9 AS tx_gas_price,                              
                                 
                             FROM (
                               SELECT *
                               FROM tx_cols
                               WHERE success IS TRUE
                               AND block_number >= {min_block}
                               AND block_number <= {max_block}
                             ) tx 
                             """)
    
    gas_price_dist = duckdb.sql(f"""
                          WITH non_null_tx_gas_price AS (
                            SELECT tx_gas_price
                            FROM gas_price_per_tx
                            WHERE tx_gas_price IS NOT NULL
                          ),

                          binned AS (
                              SELECT 
                                  FLOOR(tx_gas_price / {bin_size}) * {bin_size} AS lower_tx_gas_price
                              FROM non_null_tx_gas_price
                          ),

                          counted AS (
                              SELECT 
                                  lower_tx_gas_price,
                                  COUNT(*) AS count_blocks
                              FROM binned
                              GROUP BY lower_tx_gas_price
                          ),

                          finalized AS (
                              SELECT 
                                  lower_tx_gas_price,
                                  CASE 
                                      WHEN lower_tx_gas_price < 5000000 
                                          THEN lower_tx_gas_price + {bin_size}
                                      ELSE lower_tx_gas_price
                                  END AS upper_tx_gas_price,
                                  count_blocks
                              FROM counted
                          )

                          SELECT *
                          FROM finalized
                          ORDER BY lower_tx_gas_price DESC;
                          """)
    
    return gas_price_dist    

In [130]:
%%time

get_gas_price_dist_per_tx("2024-01-01", "2025-01-01", 100, y3).df()

CPU times: user 31 s, sys: 939 ms, total: 31.9 s
Wall time: 2.44 s


,lower_tx_gas_price,upper_tx_gas_price,count_blocks
0,13731200.0,13731200.0,1
1,1648800.0,1648900.0,1
2,1092100.0,1092200.0,1
3,842700.0,842800.0,1
4,820800.0,820900.0,1
...,...,...,...
573,400.0,500.0,84448
574,300.0,400.0,155132
575,200.0,300.0,389682
576,100.0,200.0,3124789


### Address level analytics

#### Calculate balance

In [147]:
def get_balance(address, t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        "from_address",
                        "to_address",
                        "gas_used",
                        "gas_price",
                        "value_f64",
                        folder = get_tx_folder(folder)
                       )
    
    match_tx = duckdb.sql(f"""    
                           SELECT 
                             (tx.value_f64 / 1e18) AS tx_eth,
                             tx.from_address,
                             tx.to_address,
                             tx.gas_used,
                             tx.gas_price
                               
                           FROM (
                             SELECT *
                             FROM tx_cols
                             WHERE success IS TRUE
                             AND block_number >= {min_block}
                             AND block_number <= {max_block}
                             AND to_address = FROM_HEX(SUBSTR('{address}', 3))
                             OR from_address = FROM_HEX(SUBSTR('{address}', 3))
                           ) tx 
                           """)
    
    sent_tx = duckdb.sql(f"""    
                         SELECT 
                           SUM(tx.tx_eth) AS total_tx_eth_sent,
                           SUM((tx.gas_used * tx.gas_price) / 1e18) AS total_fee_paid
                           
                         FROM (
                           SELECT *
                           FROM match_tx
                           WHERE from_address = FROM_HEX(SUBSTR('{address}', 3))
                         ) tx
                         """).fetchone()
    
    received_tx = duckdb.sql(f"""    
                             SELECT 
                               SUM(tx.tx_eth) AS total_tx_eth_received
                                 
                             FROM (
                               SELECT *
                               FROM match_tx
                               WHERE to_address = FROM_HEX(SUBSTR('{address}', 3))
                             ) tx
                             """).fetchone()
       
    return received_tx[0] - (sent_tx[0] + sent_tx[1])

In [24]:
# %%time

# get_balance("0x125ee0DC74121d4d61Dcbb65b7d10d233658bc38", 
#             "2022-01-01",
#             "2025-01-01",
#             y3)

# CPU times: user 2min 42s, sys: 3min 7s, total: 5min 49s
# Wall time: 6min 12s

# 0.44563283536997333

In [149]:
def get_balance_opt(address, t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "success",
                        "from_address",
                        "to_address",
                        "gas_used",
                        "gas_price",
                        "value_f64",
                        folder = get_tx_folder(folder)
                       )
    
    match_tx = duckdb.sql(f"""
                            SELECT
                              (value_f64 / 1e18) AS tx_eth,
                              gas_used,
                              gas_price,
                              from_address = FROM_HEX(SUBSTR('{address}', 3)) AS is_sender,
                              to_address = FROM_HEX(SUBSTR('{address}', 3)) AS is_receiver
                            FROM tx_cols
                            WHERE success IS TRUE
                              AND block_number BETWEEN {min_block} AND {max_block}
                              AND (
                                to_address = FROM_HEX(SUBSTR('{address}', 3)) OR
                                from_address = FROM_HEX(SUBSTR('{address}', 3))
                              )
                           """)

    
    summary = duckdb.sql("""
                            SELECT
                              SUM(CASE WHEN is_sender THEN tx_eth ELSE 0 END) AS total_sent,
                              SUM(CASE WHEN is_sender THEN (gas_used * gas_price) / 1e18 ELSE 0 END) AS total_fee,
                              SUM(CASE WHEN is_receiver THEN tx_eth ELSE 0 END) AS total_received
                            FROM match_tx
                        """).fetchone()
    
    balance = summary[2] - (summary[0] + summary[1])

       
    return balance

In [150]:
%%time

get_balance_opt("0x125ee0DC74121d4d61Dcbb65b7d10d233658bc38", 
            "2022-01-01",
            "2025-01-01",
            y3)

CPU times: user 2min 30s, sys: 1min 31s, total: 4min 1s
Wall time: 2min 57s


0.44563283536997333

#### Get unique addresses per year

In [6]:
def get_unique_add_count_per_year(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "to_address",
                        "from_address",
                        "success",
                        folder = get_tx_folder(folder)
                       )
    
    count_per_year = duckdb.sql(f"""
                                WITH base AS (
                                    SELECT 
                                      EXTRACT(year FROM b.timestamp) AS year, 
                                      tx.from_address AS address
                                    FROM tx_cols tx
                                    INNER JOIN blocks b 
                                      ON tx.block_number = b.block_number
                                    WHERE tx.success IS TRUE
                                      AND tx.block_number BETWEEN {min_block} AND {max_block}

                                    UNION ALL

                                    SELECT 
                                      EXTRACT(year FROM b.timestamp) AS year, 
                                      tx.to_address AS address
                                    FROM tx_cols tx
                                    INNER JOIN blocks b
                                      ON tx.block_number = b.block_number
                                    WHERE tx.success IS TRUE
                                      AND tx.block_number BETWEEN {min_block} AND {max_block}
                                ),

                                unique_addresses_per_year AS (
                                    SELECT DISTINCT year, address
                                    FROM base
                                )

                                SELECT 
                                    year,
                                    COUNT(*) AS total_unique_addresses
                                FROM unique_addresses_per_year
                                GROUP BY year
                                ORDER BY year
                                """)      
       
    return count_per_year

In [7]:
%%time

get_unique_add_count_per_year("2024-01-01", "2025-12-31", y3).df()

CPU times: user 11min 12s, sys: 2min 32s, total: 13min 45s
Wall time: 2min 40s


,year,total_unique_addresses
0,2024,56286375
1,2025,15513105


#### Get new addresses per year

In [30]:
def get_new_add_per_year_1(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "to_address",
                        "from_address",
                        "success",
                        folder = get_tx_folder(folder)
                       )
    
    new_per_year = duckdb.sql(f"""
                               WITH base AS (
                                   SELECT DISTINCT
                                     EXTRACT(year FROM b.timestamp) AS year, 
                                     tx.from_address AS address
                                   FROM tx_cols tx
                                   INNER JOIN blocks b 
                                     ON tx.block_number = b.block_number
                                   WHERE tx.success IS TRUE
                                     AND tx.block_number BETWEEN {min_block} AND {max_block}
                                     
                                   UNION ALL
                                   
                                   SELECT DISTINCT
                                     EXTRACT(year FROM b.timestamp) AS year, 
                                     tx.to_address AS address
                                   FROM tx_cols tx
                                   INNER JOIN blocks b 
                                     ON tx.block_number = b.block_number
                                   WHERE tx.success IS TRUE
                                     AND tx.block_number BETWEEN {min_block} AND {max_block}
                               ),

                               unique_addresses_per_year AS (
                                    SELECT DISTINCT year, address
                                    FROM base
                               ),

                               new_addresses AS (
                                   SELECT 
                                       a1.year,
                                       COUNT(*) AS new_unique_addresses
                                   FROM unique_addresses_per_year a1
                                   LEFT JOIN unique_addresses_per_year a2
                                     ON a1.address = a2.address 
                                     AND a2.year < a1.year
                                   WHERE a2.address IS NULL
                                   GROUP BY a1.year
                               )

                               SELECT 
                                   a.year,
                                   a.new_unique_addresses,
                               FROM new_addresses a
                               ORDER BY a.year;
                               """)
       
    return new_per_year

In [31]:
%%time

get_new_add_per_year_1("2023-01-01", "2024-12-31", y3).df()

CPU times: user 19min 59s, sys: 4min 58s, total: 24min 57s
Wall time: 6min 35s


,year,new_unique_addresses
0,2023,41780046
1,2024,44318145


In [34]:
def get_new_add_per_year_2(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "to_address",
                        "from_address",
                        "success",
                        folder = get_tx_folder(folder)
                       )
    
    new_per_year = duckdb.sql(f"""
                               WITH base AS (
                               SELECT 
                                 EXTRACT(year FROM b.timestamp) AS year, 
                                 tx.from_address AS address
                               FROM tx_cols tx
                               INNER JOIN blocks b 
                                 ON tx.block_number = b.block_number
                               WHERE tx.success IS TRUE
                                 AND tx.block_number BETWEEN {min_block} AND {max_block}
                                 
                               UNION ALL
                               
                               SELECT 
                                 EXTRACT(year FROM b.timestamp) AS year, 
                                 tx.to_address AS address
                               FROM tx_cols tx
                               INNER JOIN blocks b 
                               ON tx.block_number = b.block_number
                               WHERE tx.success IS TRUE
                                 AND tx.block_number BETWEEN {min_block} AND {max_block}
                               ),

                               earliest_year_per_address AS (
                                 SELECT address, MIN(year) AS first_seen_year
                                 FROM (
                                   SELECT DISTINCT address, year
                                   FROM base
                                 )
                                 GROUP BY address
                                ),                              

                              new_counts AS (
                                SELECT 
                                  first_seen_year AS year, 
                                  COUNT(*) AS new_unique_addresses
                                FROM earliest_year_per_address
                                GROUP BY first_seen_year
                              )

                              SELECT 
                                a.year,
                                a.new_unique_addresses                                
                              FROM new_counts a
                              ORDER BY a.year;
                              """)
       
    return new_per_year

In [36]:
# %%time

# get_new_add_per_year_2("2023-01-01", "2024-12-31", y3).df()

#### Get top address activity

In [43]:
def get_top_add_act(t_start, t_end, folder):
    
    blocks = get_blocks_filtered(t_start, t_end, get_blocks_folder(folder))
    min_block, max_block = get_blocks_min_max(blocks)

    tx_cols = ingest_pq("block_number",
                        "to_address",
                        "from_address",
                        "success",
                        folder = get_tx_folder(folder)
                       )
    
    top_add_count = duckdb.sql(f"""
                               WITH base AS (
                               SELECT 
                                 tx.from_address AS address
                               FROM tx_cols tx
                               WHERE tx.success IS TRUE
                                 AND tx.block_number BETWEEN {min_block} AND {max_block}
                                 
                               UNION ALL
                               
                               SELECT 
                                 tx.to_address AS address
                               FROM tx_cols tx
                               WHERE tx.success IS TRUE
                                 AND tx.block_number BETWEEN {min_block} AND {max_block}
                               )

                              SELECT 
                                '0x' || TO_HEX(address) AS address_hex,
                                COUNT(*) AS appearance_count
                              FROM base
                              GROUP BY address_hex
                              ORDER BY appearance_count DESC
                              LIMIT 10;
                              """)
       
    return top_add_count

In [45]:
%%time

get_top_add_act("2024-01-01", "2024-12-31", y3).df()

CPU times: user 4min 54s, sys: 53.9 s, total: 5min 48s
Wall time: 41.3 s


,address_hex,appearance_count
0,0xDAC17F958D2EE523A2206206994597C13D831EC7,39350759
1,0x3FC91A3AFD70395CD496C647D5A6CC9D4B2B7FAD,24007267
2,0xA0B86991C6218B36C1D19D4A2E9EB0CE3606EB48,12481257
3,0x7A250D5630B4CF539739DF2C5DACB4C659F2488D,5040506
4,0x80A64C6D7F12C47B7C66C5B4E20E72BC1FCD5D9E,4327661
5,0x3328F7F4A1D1C57C35DF56BBF0C9DCAFCA309C49,4248015
6,0xF89D7B9C864F589BBF53A82105107622B35EAA40,4183785
7,0x28C6C06298D514DB089934071355E5743BF21D60,3868752
8,0x974CAA59E49682CDA0AD2BBE82983419A2ECC400,3455697
9,0x1111111254EEB25477B68FB85ED929F73A960582,3436931
